# 模型构造

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

## 一. 层和块

### 1. 默认的顺序结构定义

In [14]:
net = nn.Sequential(
    nn.Linear(20, 256),
    nn.ReLU(),
    nn.Linear(256, 10),
)

In [15]:
X = torch.rand(2, 20)
net(X)

tensor([[ 0.0866,  0.1182,  0.0123,  0.1591,  0.0134, -0.1680, -0.0875, -0.2331,
         -0.2573,  0.0870],
        [ 0.0538,  0.0031, -0.1518,  0.1584,  0.0683, -0.1116,  0.1043, -0.1523,
         -0.2418,  0.0827]], grad_fn=<AddmmBackward0>)

In [6]:
print(net)

Sequential(
  (0): Linear(in_features=20, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


### 2. 自定义块 

In [17]:
class MLP(nn.Module):
    def __init__(self,):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.relu = nn.ReLU()
        self.out = nn.Linear(256, 10)
    
    def forward(self, x):
        return self.out(self.relu((self.hidden(x))))
net = MLP()
net(X)

tensor([[ 0.1297,  0.0201, -0.2489,  0.1780, -0.0096,  0.0654,  0.1661,  0.0832,
          0.1015,  0.1515],
        [ 0.1726,  0.0630, -0.1884,  0.2320,  0.0986, -0.0651,  0.1953, -0.0137,
          0.0868,  0.0369]], grad_fn=<AddmmBackward0>)

In [18]:
net

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (relu): ReLU()
  (out): Linear(in_features=256, out_features=10, bias=True)
)

### 3. 实现顺序块 Sequential

In [62]:
class MySequential(nn.Module):
    def __init__(self, *args) -> None:
        super().__init__()
        for idx, block in enumerate(args):
            self.add_module(f"({idx+1}){block.__class__.__name__}", block)
            # self._modules[f"{}"] = block
    
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

In [63]:
net = MySequential(
    nn.Linear(20, 256),
    nn.ReLU(),
    nn.Linear(256, 10),
)
net

MySequential(
  ((1)Linear): Linear(in_features=20, out_features=256, bias=True)
  ((2)ReLU): ReLU()
  ((3)Linear): Linear(in_features=256, out_features=10, bias=True)
)

### 4. 实现自定义的神经网络

In [64]:
class FixedHiddenMLP(nn.Module):

    def __init__(self, ):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
    
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()
net = FixedHiddenMLP()
net(X)

tensor(-0.1893, grad_fn=<SumBackward0>)

### 5. 混合搭配各种组合块方法

In [66]:
class NestMLP(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(20, 64), nn.ReLU(),
            nn.Linear(64, 32), nn.ReLU()
        )
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))
chimera = nn.Sequential(
    NestMLP(),
    nn.Linear(16, 20),
    FixedHiddenMLP()
)
chimera(X)

tensor(-0.2265, grad_fn=<SumBackward0>)